# Introduction to NeoRV32OnPynq


## Aims

* Introduce the compiler magic provided by NeoRV32OnPynq, and how to use it. 
* Introduce the NeoRV32OnPynq overlay GUI

# Introducing the Compiler Magic


This section will introduce the compiler magic provided by NeoRV32OnPynq, riscvc.
This magic enables C programs to be written in any Jupiter Notebook cell, and then compiled simply by running said cell
Before this magic can be used it must be loaded, which is done by the line below

In [ ]:
%load_ext NeoRV32OnPynq

The compiler magic is called using <code> %%riscvc <i>progam_name</i> </code>. 
The compiled program can be found in "programs/<i>progam_name</i>/<i>progam_name</i>.bin", where "programs" is a folder in the same location as the notebook in which the compiler magic was run. 

The "programs" folder, and its children (each named for the program they contain), serve as a way to  prevent different programs from inferring with each other. Each child folder contains not only the compiled program (.bin), but also any intermedate files (.c, .o), and some files useful for debugging (.asm, .coe).

<div class="alert alert-box alert-info">
There is no need to worry about creating a "programs" folder, or any of its children, as the compiler magic handles this as needed.
</div>

<div class="alert alert-box alert-info">
The compiler magic is a cell magic, meaning that it needs to be the first statement in a cell, and the rest of the cell's content will be interpreted by the magic.
</div>

## Compiler Magic Example

Below is an example of how to use the compiler magic.
Upon running the cell below a number of effects should be seen:
* If there wasn't already a "programs" folder, one will have been created
* If there wasn't already a "programs\flash_LEDs" folder, one will have been created
* A makefile was added to "programs\flash_LEDs"
* "programs\flash_LEDs\flash_LEDs.c" was (re)created and contains everything but the first line of the compiler magic cell 
* "programs\flash_LEDs\flash_LEDs.c.o" was (re)created
* "programs\flash_LEDs\flash_LEDs.bin" was (re)created
* "programs\flash_LEDs\flash_LEDs.asm" was (re)created
* "programs\flash_LEDs\flash_LEDs.coe" was (re)created


In [ ]:
%%riscvc flash_LEDs

// Define volatile pointer to LED GPIO handler
int volatile * const LEDs = (int*) 0x40020000;

// Define outsets for LED GPIO handler's regmap
const int value = 0;
const int tristate = 1;

int main()
{
    // Set all LEDs GPIO as outputs
    LEDs[tristate] = 0;
    
    // Set the LEDs in off state
    LEDs[value] = 0;

    // Toggle the LEDs forever
    while(1)
    {
        LEDs[value] = ~LEDs[value];
    }

    return 0;
}



## Compiler Magic Example 2


A notebook is not restricted to compiling a single program, below is an example of the compiler magic being used to compile another program, the same set of effects should be seen upon running this cell.

In [ ]:
%%riscvc LED_counter

// Define int32_t
#include <stdint.h>

// Define volatile pointer to LED GPIO handler
int32_t volatile * const LEDs = (int32_t*) 0x40020000;

// Define outsets for LED GPIO handler's regmap
const int32_t value = 0;
const int32_t tristate = 1;

int main()
{
    // Set all LEDs GPIO as outputs
    LEDs[tristate] = 0;

    // Toggle the LEDs forever
    while(1)
    {
        for(int32_t i=0; i < 16; i++)
        {
            LEDs[value] = i;
        }
    }

    return 0;

# Interacting with the Processor


Now that we have some programs compiled with the compiler magic, lets look at how to interact with the the NeoRV processor.
All interactions with the processor are done through the `NeoRV32` class within the `NeoRV32OnPynq` module.
The code below first imports the `NeoRV32` class and then used it to setup the board.

In [1]:
from NeoRV32OnPynq import NeoRV32
neorv = NeoRV32()

The recommended method of interacting with the processor is via the GUI provided; which consists of 3  parts, each handling a different stage/type of interaction,
Each part will be covered in their own section, but below is an overview of each.
* Program loading, this section handles the selection and loading of programs to run on the processor
* Execution control, this section controls when the processor can executes instruction, providing stepping, continous, and run until condition options
* Internal Inspection, this sectio display a range of signals internal to the NeoRV32, aiding in debugging and/or tracking of the instructions though the processor

## Program Loading


The program loading section of the GUI is responsbile for enabling a user to select any of the programs found in "programs" and loading them to be run by the NeoRV32. 
The accessed via the `load_program` function of the `NeoRV32` class as shown below.

In [2]:
neorv.load_program()

Can't Write, no program selected
Can't Write, no program selected
Can't Write, no program selected


There are four points of interest on this GUI:-
* The refresh programs button, when pressed this button rescans`programs` and updates the GUI's list of available programs e.g. removing any  that have been deleted or adding newly compiled programs
* The program select dropdown, this dropdown lists all programs known to the GUI letting you select which you wish to load on to the overlay
* The load program button, when clicked this button will read the binary of the program selected using the program select dropdown and load it into the overlay BRAM and reset the pc to the start of the program
* The last loaded field, this field shows the name of the last program loaded by the GUI. This will normal be the program currently running on the board, assuming the board has remained connected and only the GUI has been used for loading programs.

## execution_control GUI

The execution_control GUI is responsible for controling the overlay's NERORV core's clock, this enables control over when the NERORV core can execute instructions. This control can vary from  running in single clock cycle steps to running the overlay until a specific position within the program is reached. The execution_control GUI is displayed using the code below

In [ ]:
 ol.execution_control()

## show_internals GUI

display a range of signals internal to the overlay, to aid in debugging and/or tracking of the instructions though the overlay

In [ ]:
ol.show_internals()